In [62]:
import numpy as np

def hh_reflection(o_vec:np.ndarray, p_vec:np.ndarray) -> np.matrix:
    v:np.ndarray
    """
    Hausehold reflection operator
    params:
        o_vec: the original vector which want to be reflected
        p_vec: the direction of reflected vector
    return:
        hh_ref: the reflection operator
    """
    n = np.size(o_vec,)
    v = o_vec - p_vec; v = v.reshape((n,1))
    return np.eye(n) - 2 * (v @ v.T) / (v.T @ v)

def eye_leftlow_cat(dim:int, mat:np.matrix) -> np.matrix:
    """
    concat an eye matrix and a matrix on the diagnal 
    input:
        mat: the matrix waiting to be concat
    output:
        mat: [[I,   0],
              [0, mat]]
    """
    if dim == 0: return mat # 若 dim = 0 返回原矩阵
    mat_dim = mat.shape[0]; 
    mattop = np.c_[np.eye(dim), np.zeros((dim, mat_dim))] # 水平拼接
    matbot = np.c_[np.zeros((mat_dim, dim)), mat]         # 水平拼接
    return np.r_[mattop, matbot]                          # 垂直拼接

def QR_factorization(mat:np.ndarray):
    """
    QR-factorization of the input matrix using Hausehold Reflection method
    input:
        mat: the matrix to be QR factorized
    output:
        Q, R
    """
    dim1 = mat.shape[1]; Q = np.eye(mat.shape[0]) # 初始化 Q 矩阵
    for i in range(dim1): 
        if i == mat.shape[0] - 1: break # 如果是方阵, 最后一个元素不动
        tmp_vec = mat[i:,i]; tmp_vec_norm = np.linalg.norm(tmp_vec, ord = 2) # 找到等待反射的向量
        reflect_target = np.zeros_like(tmp_vec, dtype = np.float32); reflect_target[0] = tmp_vec_norm # 构造 [|a|,0,,...] 向量
        reflect_operator = hh_reflection(tmp_vec, reflect_target) # 计算反射矩阵
        reflect_operator = eye_leftlow_cat(dim = i, mat = reflect_operator) # 反射矩阵分块化
        Q = Q @ reflect_operator # Q 矩阵累乘
        mat = reflect_operator @ mat # 更新 R 矩阵
    return Q, mat

def QR_factorization_cpivot(mat:np.ndarray):
    """
    QR-factorization of the input matrix using Hausehold Reflection method and column pivot
    input:
        mat: the matrix to be QR factorized
    output:
        Q, R, P.Transpose()
        P is the permutation matrix which indicates: AP = QR
    """
    def pair_pmat(n, ind1, ind2):
        """
        Calculate the pair permutation matrix which switch the column ind1 and ind2
        params:
            n: the dimension of mat
            ind1, ind2: the index to switch
        return:
            P: the permutation mat
        """
        P = np.eye(n)
        P[ind1, ind1] = 0; P[ind1, ind2] = 1
        P[ind2, ind2] = 0; P[ind2, ind1] = 1
        return P

    dim1 = mat.shape[1]; Q, P = np.eye(mat.shape[0]), np.eye(mat.shape[1]) # 初始化 Q, P 矩阵
    for i in range(dim1): 
        if i == mat.shape[0] - 1: break # 如果是方阵, 最后一个元素不动
        pivot_ind = np.argmax(np.abs(mat[i,:])) # 主元位置
        pivot = mat[i, pivot_ind]; P_tmp = pair_pmat(dim1, i, pivot_ind)
        mat = mat @ P_tmp
        tmp_vec = mat[i:,i]; tmp_vec_norm = np.linalg.norm(tmp_vec, ord = 2) # 找到等待反射的向量
        reflect_target = np.zeros_like(tmp_vec, dtype = np.float32); reflect_target[0] = tmp_vec_norm # 构造 [|a|,0,,...] 向量
        reflect_operator = hh_reflection(tmp_vec, reflect_target) # 计算反射矩阵
        reflect_operator = eye_leftlow_cat(dim = i, mat = reflect_operator) # 反射矩阵分块化
        Q = Q @ reflect_operator # Q 矩阵累乘
        mat = reflect_operator @ mat # 更新 R 矩阵
        P = P @ P_tmp # 更新 P 矩阵
    return Q, mat, P.T


In [43]:
QR_factorization(np.array([[1,4,5],[5,7,6],[44,9,11]]))

(array([[ 0.02257621,  0.53672414, -0.8434557 ],
        [ 0.11288091,  0.83690717,  0.53557846],
        [ 0.99335201, -0.10730138, -0.04169172]]),
 array([[ 4.42944692e+01,  9.82063930e+00,  1.17170386e+01],
        [-6.88968549e-07,  7.03953434e+00,  6.52474855e+00],
        [ 1.08270600e-06,  9.92445787e-07, -1.46241660e+00]]))

In [63]:
QR_factorization_cpivot(np.array([[1,4],[5,7],[44,9]]))

(array([[ 0.33104236, -0.42307606,  0.84345575],
        [ 0.57932412, -0.61444232, -0.53557837],
        [ 0.7448453 ,  0.66593339,  0.04169173]]),
 array([[ 1.20830460e+01,  3.60008561e+01],
        [ 8.93127502e-09,  2.58057815e+01],
        [-1.78056284e-08,  8.29364610e-08]]),
 array([[0., 1.],
        [1., 0.]]))

In [50]:
from numpy.linalg  import cholesky
from numpy.polynomial.polynomial import polyval

m = 21; n = 12; eps = 1e-10; c0 = np.ones(n)
t = np.linspace(0, 1, m); y0 = polyval(x = t, c = c0)
y0 = y0 + (np.random.uniform(low = 0, high = 1, size = y0.shape) * 2 - 1) * eps


# Van der Monde 矩阵
Vo = np.vander(t, N = n, increasing=True)
Vo.shape

(21, 12)

In [51]:
# 法线方程部分
L = cholesky(Vo.T @ Vo) # cholesky factorization
b = Vo.T @ y0           # 法线方程右端
tmp_c = np.linalg.solve(L, b); c_normal = np.linalg.solve(L.T, tmp_c)
error_normal = np.linalg.norm(c_normal - c0)
error_normal

0.14059603340600213

In [61]:
# QR 分解部分
Q, R = QR_factorization(Vo) # QR factorization without column pivot
R = R[:R.shape[1],:]        # 丢掉 0 行
b = (Q.T @ y0)[:R.shape[1]] # 等式右侧也同样去掉相同的行
c_qr = np.linalg.solve(R, b)
error_qr = np.linalg.norm(c_qr - c0)
error_qr

0.000630076542429547

In [64]:
# cpivot-QR 分解部分
Q, R, P = QR_factorization_cpivot(Vo) # QR factorization with column pivot
R = R[:R.shape[1],:]        # 丢掉 0 行
b = (Q.T @ y0)[:R.shape[1]] # 等式右侧也同样去掉相同的行
c_qr = np.linalg.solve(R, b)
c_qr_cpivot = P.T @ c_qr
error_qr_pivot = np.linalg.norm(c_qr_cpivot - c0)
error_qr_pivot

0.0006198408113126203